# Lab | Chains in LangChain

## Outline

* LLMChain
* Sequential Chains
  * SimpleSequentialChain
  * SequentialChain
* Router Chain

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')
HUGGINGFACEHUB_API_TOKEN = os.getenv('HUGGINGFACEHUB_API_TOKEN')

In [3]:
!pip install pandas

In [4]:
import pandas as pd
df = pd.read_csv('./data/Data.csv')

In [5]:
df.head()

,Product,Review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld\n,I loved this product. But they only seem to l...


## LLMChain

In [6]:
!pip install langchain_community

In [7]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [8]:
#Replace None by your own value and justify
llm = ChatOpenAI(temperature=0.7)


A temperature of 0.7 was chosen to balance creativity and coherence in the model's responses. This value encourages the language model to generate more diverse and interesting outputs than a lower temperature (e.g., 0 or 0.2), while still maintaining logical and relevant answers. It is a common default for tasks where some variability is desired without sacrificing too much reliability or factual accuracy.

In [9]:
prompt = ChatPromptTemplate.from_template( #Write a query that would take a variable to describe any product
"Describe the product: {product}"
)

In [10]:

chain = LLMChain(llm=llm, prompt=prompt)

/var/folders/s7/8db2fs716sj67tmf9cc2npbm0000gn/T/ipykernel_7400/546483037.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=prompt)


In [11]:
# Select a product type to be described
product = df.Product[0]
chain.run(product)

/var/folders/s7/8db2fs716sj67tmf9cc2npbm0000gn/T/ipykernel_7400/439173260.py:3: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  chain.run(product)


"A Queen Size Sheet Set typically includes a fitted sheet, a flat sheet, and two pillowcases designed specifically to fit a queen-sized bed. These sheet sets are made from a variety of materials such as cotton, microfiber, or bamboo, and come in a range of colors and patterns to suit different styles and preferences. The sheets are soft, comfortable, and durable, providing a cozy and luxurious feel for a restful night's sleep."

## SimpleSequentialChain

In [12]:
from langchain.chains import SimpleSequentialChain

In [13]:
llm = ChatOpenAI(temperature=0.9)

# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
    #Repeat the initial query or create a new query that would feed into the second prompt
    "Describe the category: {category}"
)

# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)

In [14]:

# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    #Write the second prompt query that takes an input variable whose input will come from the previous prompt"
	"Write a creative product description based on the following category description: {category}"
)
# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [15]:
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                             verbose=True
                                            )

In [16]:
overall_simple_chain.run(product)



> Entering new SimpleSequentialChain chain...
Queen size sheet sets are bedding sets designed to fit a queen size mattress, which typically measures 60 inches wide and 80 inches long. These sets usually include a fitted sheet, a flat sheet, and one or two pillowcases, all made from a variety of materials such as cotton, linen, or microfiber. Queen size sheet sets come in a range of colors, patterns, and thread counts to suit different preferences and needs. They are a popular choice for many households due to their versatility and ability to provide a comfortable and stylish bedding option for queen size beds.
Upgrade your bedroom decor with our luxurious queen size sheet sets. Made from high-quality materials such as cotton, linen, or microfiber, these sets are designed to provide the perfect combination of comfort and style. Our sets include a fitted sheet, a flat sheet, and one or two pillowcases, ensuring your queen size mattress is fully covered and beautifully coordinated.

Cho

'Upgrade your bedroom decor with our luxurious queen size sheet sets. Made from high-quality materials such as cotton, linen, or microfiber, these sets are designed to provide the perfect combination of comfort and style. Our sets include a fitted sheet, a flat sheet, and one or two pillowcases, ensuring your queen size mattress is fully covered and beautifully coordinated.\n\nChoose from a wide range of colors, patterns, and thread counts to find the perfect set to match your existing decor or to add a pop of color to your bedroom. Whether you prefer a classic white sheet set for a clean and crisp look, or a bold pattern to make a statement, we have something for everyone.\n\nExperience the ultimate in comfort and style with our queen size sheet sets. Upgrade your bedding today and transform your bedroom into a relaxing oasis.'

**Repeat the above twice for different products**

## SequentialChain

In [17]:
from langchain.chains import SequentialChain

In [23]:
llm = ChatOpenAI(temperature=0.9)


first_prompt = ChatPromptTemplate.from_template(
  #This prompt should translate a review
  "Translate the following review to French:\n{review}"
)

chain_one = LLMChain(llm=llm, prompt=first_prompt, 
                     output_key="French" #Give a name to your output
                    )


In [19]:
second_prompt = ChatPromptTemplate.from_template(
    #Write a promplt to summarize a review
	"Summarize the following review in 1-2 sentences:\n{review}"
)

chain_two = LLMChain(llm=llm, prompt=second_prompt, 
                     output_key="Summary" #give a name to this output
                    )


In [21]:
# prompt template 3: translate to english or other language
third_prompt = ChatPromptTemplate.from_template(
    "Translate the following review to English:\n{review}"
)
# chain 3: input= Review and output= language
chain_three = LLMChain(llm=llm, prompt=third_prompt,
                       output_key="Language"
                      )


In [ ]:

# prompt template 4: follow up message that take as inputs the two previous prompts' variables
fourth_prompt = ChatPromptTemplate.from_template(
		"Write a follow-up message to the customer based on the following summary and English translation of their review:\nSummary: {Summary}\nEnglish Review: {Language}"
)
chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key="Followup"
                     )


In [26]:
# overall_chain: input= Review 
# and output= English_Review,summary, followup_message
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["review"],
    output_variables=["French", "Summary", "Language", "Followup"],
    verbose=True
)

In [27]:
review = df.Review[5]
overall_chain(review)

/var/folders/s7/8db2fs716sj67tmf9cc2npbm0000gn/T/ipykernel_7400/1992003631.py:2: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  overall_chain(review)




> Entering new SequentialChain chain...

> Finished chain.


{'review': "Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\nVieux lot ou contrefaçon !?",
 'French': "I find the taste mediocre. The foam doesn't last, it's weird. I buy the same ones in stores and the taste is much better... Old batch or counterfeit!?",
 'Summary': "Le reviewer trouve le goût de la mousse médiocre et suspecte d'être contrefaite, car il a acheté les mêmes dans le commerce avec un meilleur goût.",
 'Language': "I find the taste mediocre. The foam doesn't hold, it's strange. I buy the same ones in stores and the taste is much better... Old batch or counterfeit!?",
 'Followup': 'Dear valued customer,\n\nThank you for sharing your feedback with us. We apologize that you found the taste of our foam to be mediocre and suspect it may be counterfeit or from an old batch. Rest assured, we take quality control very seriously and will investigate this matter further to ensure that our products

**Repeat the above twice for different products or reviews**

In [28]:
review = df.Review[2]
overall_chain(review)



> Entering new SequentialChain chain...

> Finished chain.


{'review': "This mattress had a small hole in the top of it (took forever to find where it was), and the patches that they provide did not work, maybe because it's the top of the mattress where it's kind of like fabric and a patch won't stick. Maybe I got unlucky with a defective mattress, but where's quality assurance for this company? That flat out should not happen. Emphasis on flat. Cause that's what the mattress was. Seriously horrible experience, ruined my friend's stay with me. Then they make you ship it back instead of just providing a refund, which is also super annoying to pack up an air mattress and take it to the UPS store. This company is the worst, and this mattress is the worst.",
 'French': "Ce matelas avait un petit trou sur le dessus (il a fallu une éternité pour trouver où il était), et les rustines fournies ne fonctionnaient pas, peut-être parce que c'est le dessus du matelas où c'est un peu comme du tissu et qu'une rustine ne collera pas. Peut-être que j'ai eu la m

In [32]:
review = df.Review[1]
overall_chain(review)



> Entering new SequentialChain chain...

> Finished chain.


{'review': 'I loved the waterproof sac, although the opening was made of a hard plastic. I don’t know if that would break easily. But I couldn’t turn my phone on, once it was in the pouch.',
 'French': "J'ai adoré le sac imperméable, même si l'ouverture était faite en plastique dur. Je ne sais pas si ça se casserait facilement. Mais je n'ai pas pu allumer mon téléphone une fois qu'il était dans la pochette.",
 'Summary': 'The reviewer liked the waterproof sac but was concerned about the durability of the hard plastic opening and found it difficult to turn their phone on once inside the pouch.',
 'Language': 'Me encantó el saco impermeable, aunque la apertura estaba hecha de plástico duro. No sé si se romperá fácilmente. Pero no pude encender mi teléfono una vez que estaba en la bolsa.',
 'Followup': "Dear valued customer,\n\nThank you for sharing your feedback with us regarding the waterproof sac. We're glad to hear that you liked the waterproof feature, but we understand your concerns

## Router Chain

In [33]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""

biology_template = """You are an excellent biologist. \
You have a deep understanding of living organisms, \
from the molecular and cellular level to entire ecosystems. \
You are skilled at observing patterns in nature, analyzing biological data, \
and explaining complex processes like evolution, genetics, physiology, and ecology. \
You can clearly communicate how life functions and adapts, \
and you make connections between different biological concepts \
to answer challenging questions.

Here is a question:
{input}"""

In [34]:
prompt_infos = [
    {
        "name": "physics", 
        "description": "Good for answering questions about physics", 
        "prompt_template": physics_template
    },
    {
        "name": "math", 
        "description": "Good for answering math questions", 
        "prompt_template": math_template
    },
    {
        "name": "History", 
        "description": "Good for answering history questions", 
        "prompt_template": history_template
    },
    {
        "name": "computer science", 
        "description": "Good for answering computer science questions", 
        "prompt_template": computerscience_template
    },
    {
        "name": "biology",
        "description": "Good for answering biology questions",
        "prompt_template": biology_template
    }
]

In [35]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate

In [36]:
llm = ChatOpenAI(temperature=0)

In [37]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain  
    
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [38]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [39]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [40]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [41]:
chain = MultiPromptChain(router_chain=router_chain, 
                         destination_chains=destination_chains, 
                         default_chain=default_chain, verbose=True
                        )

/var/folders/s7/8db2fs716sj67tmf9cc2npbm0000gn/T/ipykernel_7400/3038952769.py:1: LangChainDeprecationWarning: Please see migration guide here for recommended implementation: https://python.langchain.com/docs/versions/migrating_chains/multi_prompt_chain/
  chain = MultiPromptChain(router_chain=router_chain,


In [42]:
chain.run("What is black body radiation?")



> Entering new MultiPromptChain chain...
physics: {'input': 'What is black body radiation?'}
> Finished chain.


"Black body radiation is the electromagnetic radiation emitted by a perfect absorber and emitter of radiation, known as a black body. A black body absorbs all radiation that falls on it and emits radiation at all wavelengths. The spectrum of black body radiation is continuous and depends only on the temperature of the black body. This phenomenon is described by Planck's law, which states that the intensity of radiation emitted by a black body at a given wavelength is proportional to the temperature of the body and the wavelength raised to the fifth power."

In [43]:
chain.run("what is 2 + 2")



> Entering new MultiPromptChain chain...
math: {'input': 'what is 2 + 2'}
> Finished chain.


'2 + 2 is equal to 4.'

In [44]:
chain.run("Why does every cell in our body contain DNA?")



> Entering new MultiPromptChain chain...
biology: {'input': 'Why does every cell in our body contain DNA?'}
> Finished chain.


'Every cell in our body contains DNA because DNA is the genetic material that carries the instructions for the development, functioning, and reproduction of all living organisms. DNA contains the information needed to build and maintain an organism, including the proteins that make up our cells and tissues. \n\nHaving DNA in every cell ensures that each cell has the necessary genetic information to carry out its specific functions and to replicate itself accurately during cell division. This ensures that the genetic information is passed on to the next generation of cells. \n\nAdditionally, DNA is constantly being used by cells to carry out processes such as protein synthesis, cell division, and repair. Having DNA in every cell allows for the coordination of these processes and ensures that the organism functions properly as a whole. \n\nIn summary, every cell in our body contains DNA because it is essential for the proper functioning and development of all living organisms.'

**Repeat the above at least once for different inputs and chains executions - Be creative!**